In [1]:
# %% Imports

import random
import csv

import numpy
import pandas
import tensorflow as tf
from keras import layers, utils, losses, metrics
import matplotlib.pyplot as pyplot
from tqdm import tqdm

from model import UNet, ClassifierNet

# Check for GPU
print(tf.config.list_physical_devices("GPU"))

WIDTH = 256
HEIGHT = 256
SIZE = (WIDTH, HEIGHT)
INPUTS_DIR = "datasets/validation/images/"
LABELS_DIR = "datasets/validation/segmentation/"
LABELS_FILE = "datasets/validation/classification.csv"
META_FILE = "datasets/validation/metadata.csv"
BATCH_SIZE = 1
SAVED_FILE = f"saved/p3/weights_{WIDTH}_{HEIGHT}.h5"

2024-01-21 03:41:28.271157: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-21 03:41:28.271197: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-21 03:41:28.271204: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-21 03:41:28.274816: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-01-21 03:41:29.637947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 03:41:29.673428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 03:41:29.673471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
target = pandas.read_csv(LABELS_FILE, header=0)
metadata = pandas.read_csv(META_FILE, header=0)
data_tb = pandas.merge(target, metadata, on="image_id")
data_tb['age_approximate'] = data_tb['age_approximate'].map(lambda x: float(x) if x != 'unknown' else -1.0)
data_tb['male'] = (data_tb['sex'] == 'male').astype(float)
data_tb['female'] = (data_tb['sex'] == 'female').astype(float)
data_tb['nevus'] = ((data_tb['melanoma'] < 0.5) & (data_tb['seborrheic_keratosis'] < 0.5)).astype(float)
metadata_tb = data_tb[['age_approximate', 'male', 'female']].to_numpy()
target_tb = data_tb[['nevus', 'melanoma', 'seborrheic_keratosis']].to_numpy()
images_tb = (INPUTS_DIR + data_tb['image_id'] + '.jpg').to_list()

In [3]:
def load_image(path):
    file = tf.io.read_file(path)
    image = tf.io.decode_image(file, channels=3, expand_animations=False)
    image = tf.image.resize(image, size=SIZE)
    return image

paths_ds = tf.data.Dataset.from_tensor_slices(images_tb)
inputs_ds = paths_ds.map(load_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).cache()
metadata_ds = tf.data.Dataset.from_tensor_slices(metadata_tb).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).cache()

2024-01-21 03:41:32.303577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 03:41:32.303686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 03:41:32.303721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 03:41:32.424694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-21 03:41:32.424770: I tensorflow/compile

In [4]:
model = ClassifierNet()
model((layers.Input((WIDTH, HEIGHT, 3)), layers.Input(3)))
model.load_weights(SAVED_FILE)

In [5]:
outputs_tb = numpy.ndarray((0, 3))
zipped = tf.data.Dataset.zip((inputs_ds, metadata_ds))
for inputs, metadata in zipped:
    inputs = inputs / 255
    outputs = model((inputs, metadata))
    outputs_tb = numpy.append(outputs_tb, outputs.numpy(), axis=0)

2024-01-21 03:41:49.742480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-01-21 03:41:49.867086: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:521] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.8
  /usr/local/cuda
  /home/luan/miniconda3/envs/luanenv/lib/python3.9/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/luan/miniconda3/envs/luanenv/lib/python3.9/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.


In [6]:
target_argmax = numpy.argmax(target_tb, axis=1)
outputs_argmax = numpy.argmax(outputs_tb, axis=1)
melanoma_P = numpy.sum(target_argmax == 1)
melanoma_TP = numpy.sum((target_argmax == 1) & (outputs_argmax == 1))
melanoma_N = numpy.sum(target_tb != 1)
melanoma_TN = numpy.sum((target_argmax != 1) & (outputs_argmax != 1))
print("Sensitivity of melanoma: " + str(melanoma_TP / melanoma_P))
print("Specificity of melanoma: " + str(melanoma_TN / melanoma_N))
print("Accuracy of melanoma: " + str((melanoma_TP + melanoma_TN) / (melanoma_P + melanoma_N)))

Sensitivity of melanoma: 0.43333333333333335
Specificity of melanoma: 0.3566666666666667
Accuracy of melanoma: 0.36363636363636365


In [7]:
target_argmax = numpy.argmax(target_tb, axis=1)
outputs_argmax = numpy.argmax(outputs_tb, axis=1)
sk_P = numpy.sum(target_argmax == 2)
sk_TP = numpy.sum((target_argmax == 2) & (outputs_argmax == 2))
sk_N = numpy.sum(target_tb != 1)
sk_TN = numpy.sum((target_argmax != 2) & (outputs_argmax != 2))
print("Sensitivity of seborrheic keratosis: " + str(sk_TP / sk_P))
print("Specificity of seborrheic keratosis: " + str(sk_TN / sk_N))
print("Accuracy of seborrheic keratosis: " + str((sk_TP + sk_TN) / (sk_P + sk_N)))

Sensitivity of seborrheic keratosis: 0.5
Specificity of seborrheic keratosis: 0.3233333333333333
Accuracy of seborrheic keratosis: 0.34502923976608185
